<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Needleman and Wunsch algorithm

## Recursive form

In [ ]:
# this is so that we can use print() in python2 like in python3
from __future__ import print_function
# with this, division will behave in python2 like in python3
from __future__ import division

### Cost functions

In order to model the insertion and substitution costs as described in video from sequence 7, we assume we have two functions:
 * `insertion_cost(base)`
 * `substitution_cost(base1, base2)`

The algorithm uses these functions, that we will be able to provide separately without the need to change the main algorithm.

### The recursive form for the algorithm

We will write a function named `needleman_wunsch_rec`. So as to avoid the creation of substrings at each recursive all, we will pass the function both the actual chains, and the corresponding indices `i` and `j`.

##### Stop condition

The recursive algorithm is about calling the function itself with smaller indices. Recursivity stops for us on **the upper and left edges of the frame**. Which means that, ifever `i` or `j` is `0`, we compute a cost as the sum of insertion costs for the non-empty string.

That is to say, if for example we have:

In [ ]:
dna1 = "ACGT"
dna2 = "GTAC"

Then for a point on the upper edge: `needleman_wunsch_rec(dna1, dna2, 2, 0)` is expected to return `insertion_cost('A') + insertion_cost('C')` 
  
and likewise on the left hand side frame, if we receive as input `i=0` et `j=2` then we must return `insertion_cost('G') + insertion_cost('T')`

##### The algorithm

One last point to notice before we go ahead is about limits. Again indices in python start at 0, and so we must tweak the formulas from the video. For example when we get as input the strings `ABC` and `AC`, we are to compute 4x3 values to fill in the form, as we can see on this figure:

![Indices for computation](media/nw-indices.png)

This is why in our case the main formula is

    cost (i, j) = min ( cost(i-1, j-1) + substitution(dna1[i-1], dna2[j-1]),
                        ...)
                        
and not `substitution(dna1[i], dna2[j])`

Here is finally the code for the recursive form of that algorithm:

In [ ]:
def needleman_wunsch_rec(dna1, dna2, i=None, j=None):
    """
    Computes the distance between two DNA chains according to the
    Needleman and Wunsch algorithm - recursive form
    
    Uses the cost functions
     * insertion_cost(base) and
     * substitution_cost(base1, base2)
     
    The caller does not specify `i` and `j`, that are used internally
    through the recursive logic
    """
    # if i and j are not specified, it means we want to
    # deal with the whole chains
    i = i if i is not None else len(dna1)
    j = j if j is not None else len(dna2)

    ### stop condition
    # upper edge
    if j == 0:
        return sum(insertion_cost(base) for base in dna1[:i])
    # left hand side edge
    elif i == 0:
        return sum(insertion_cost(base) for base in dna2[:j])
        
    # in the general case, we can apply the forumla as-is
    return min(
        # substitution
        needleman_wunsch_rec(dna1, dna2, i-1, j-1) + substitution_cost(dna1[i-1], dna2[j-1]),
        # insertion
        needleman_wunsch_rec(dna1, dna2, i, j-1) + insertion_cost(dna2[j-1]),
        # insertion
        needleman_wunsch_rec(dna1, dna2, i-1, j) + insertion_cost(dna1[i-1]))

### A simple version for the cost functions

Before we can use this function, we need to provide an implementation for the cost functions; here is the simplest version, where insertion always costs `1`, and substitution costs `1` in case of an actual change, and `0` otherwise:

In [ ]:
# the simplest insertion cost function
def insertion_cost(base):
    return 1

In [ ]:
# the simplest possible substitution cost
def substitution_cost(base1, base2):
    return 1 if base1 != base2 else 0

In [ ]:
# examples
print("insertion", insertion_cost('A'))
# this is free 
print("replacement with the same letter", substitution_cost('A', 'A'))
# this is not
print("actual replacement", substitution_cost('A', 'T'))

### The algorithm in action

Let us start with a very small example where both chains are almost identical:

In [ ]:
needleman_wunsch_rec("ACTG", "ACTC")

which in this specific case is also Hamming's distance. Now in order to appreciate how powerful the algorithm is, let us consider a case where one base has been inserted:

In [ ]:
needleman_wunsch_rec("ACGTAGC", 
                     "ACTGTAGC")
#                       ^           

However this version is **very inefficient**. If we try to use it with chains a little longer, see how long it takes already with chains that are in the 10 bases long (allow for a few seconds):

In [ ]:
needleman_wunsch_rec("ACTGCCAAC", "ACTGCGCAAC")

### Complexity analysis

You probably already figured it out by yourself at this point, but that algorithm is totally unusable in this state, becaus it is stupid, in that computes the same values over and over again. As a result, its complexity is **exponential**. I suggest a revised, simplified version, but instrumented so that we see more clearly what is going on:

In [ ]:
counter = 0

# a similar, but verbose, version
def nw(dna1, dna2, i, j):
    global counter
    counter += 1
    print("couple ({}, {})".format(i, j))
    if i == 0 or j == 0:
        return (i+j)
    return min(
        # en diagonale
        nw(dna1, dna2, i-1, j-1) + (dna1[i] != dna2[j]),
        # a gauche
        nw(dna1, dna2, i, j-1) + 1,
        # en montant
        nw(dna1, dna2, i-1, j) + 1)

In [ ]:
counter = 0
nw("ACGT", "ACGT", 2, 2)
print("nw was called {} times".format(counter))

So as you can see the function gets called **three times**  with arguments `(0, 1)`. If you try to call `nw` with arguments `3, 3` you will bump from 19 to .. 94 calls !

In [ ]:
counter = 0
nw("ACGT", "ACGT", 3, 3)
print("nw was called {} times".format(counter))

This exponential complexity makes the algorithm prohibitively expensive, and calls for the iterative version, that comes with a more affordable complexity.